## 04/02 ~ continuing work with py files

In [1]:
from src.data_cleaning import data_labeling
from src.data_cleaning import feature_engineering
from src.language_processing import to_tfidf

In [2]:
import pandas as pd
#pd.set_option('display.max_columns', 100)

In [3]:
df = data_labeling('data/data_prepping_zeros_index.xlsx',
              'data/smalleyeball_partone.xlsx',
              'data/smalleyeball_parttwo.xlsx')

/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [4]:
df2 = feature_engineering(df)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [5]:
df2.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,adobe sign,listed_count,description,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,True,1,There's no substitute for experience! Call 609...,6750,108,1,12,-0.049140,0.998792,False
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,True,14,We provide an affordable way to have a wonderf...,5018,47,2,25,-0.501007,0.865443,False
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,True,0,,29,60,10,52,-0.599548,0.800339,True
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,True,2,,5332,66,0,182,-0.325843,0.945424,True
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,True,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348,-0.098162,0.995170,False


In [6]:
df = df2

In [7]:
text_tfidf_matrix = to_tfidf(df,'text')

In [8]:
desc_tfidf_matrix = to_tfidf(df,'description')

In [14]:
text_tfidf_matrix.mean()

0.00022003398980305908

In [15]:
desc_tfidf_matrix.mean()

0.000551184644606618

# Finally starting modeling!

## Approaching way

1. numerical predictors will be modeled with logistic regression, random forest, gradient boosting

2. TFIDF matrix predictors will be modeled by Naive Bayes. Currently, as I'm testing with local computer, I reduced down the number of max features to 1000. However,if I start cloud computing (Amazon AWS), I will increase the number of max features

3. Ultimately, I will ensemble three different models - one based on numerical predictors and two based on post text TFIDF matrix and description text TFIDF matrix. After evaluating all model, I will find a best ratio for the weight of each model to be ensembled.

## Spam_bot

### Logistic Regression

In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import numpy as np
from sklearn.linear_model import LogisticRegression

In [10]:
from sklearn.model_selection import train_test_split

spam bot is the minimum level of spam to work with. I will start with this one.

In [12]:
df.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'description', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none',
       'text_without_accent', 'text_word_tokens', 'processed_text', 'stemmed',
       'ngrammed'],
      dtype='object')

In [20]:
df_model_training = df.iloc[:16000]
df_final_validation = df.iloc[16000:]

In [21]:
y = df_model_training['spam_bot']
X = df_model_training[['Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none']]

In [22]:
X.head()

,Docusign,onespan,signnow,adobe sign,listed_count,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none
0,False,False,False,True,1,6750,108,1,12,-0.049140,0.998792,False
1,False,False,False,True,14,5018,47,2,25,-0.501007,0.865443,False
2,False,False,False,True,0,29,60,10,52,-0.599548,0.800339,True
3,False,False,False,True,2,5332,66,0,182,-0.325843,0.945424,True
4,False,False,False,True,8,1337,225,19,348,-0.098162,0.995170,False


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [26]:
logistic_model = LogisticRegression()

In [27]:
logistic_model.fit(X_train,y_train)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
y_predict = logistic_model.predict(X_test)

In [63]:
y_pred = logistic_model.predict_proba(X_test)

In [61]:
sum(y) / len(y)

0.727125

In [72]:
np.argmax(logistic_model.coef_)
X.columns[3]

'adobe sign'

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred))
print(roc_auc_score(y_test,y_predict))

0.98825
[[1010   47]
 [   0 2943]]
0.11483660686215527
0.9777672658467361


In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
rf_model = RandomForestClassifier()

In [46]:
rf_model.fit(X_train,y_train)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
y_predict = logistic_model.predict(X_test)

In [66]:
y_pred_rf = rf_model.predict_proba(X_test)

In [67]:
print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred_rf))
print(roc_auc_score(y_test,y_predict))

0.98825
[[1010   47]
 [   0 2943]]
0.12788181628188067
0.9777672658467361


In [52]:
from sklearn.ensemble import GradientBoostingClassifier

In [53]:
gb_model = GradientBoostingClassifier()

In [54]:
gb_model.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [55]:
y_predict = gb_model.predict(X_test)

In [68]:
y_pred_gb = gb_model.predict_proba(X_test)

In [69]:
print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred_gb))
print(roc_auc_score(y_test,y_predict))

0.98825
[[1010   47]
 [   0 2943]]
0.04897810445502796
0.9777672658467361


This is understandable performance - every automation post is from Adobe Sign and there is a boolean column to see if the post is about Adobe Sign or not.

As the task was easy, good performance was understandable.

In [73]:
X.columns[np.argmax(logistic_model.coef_)]

'adobe sign'